In [1]:
import requests as rq
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import math
from random import randint
import json
import pickle
import time
from pprint import pprint
from lxml import html
import requests
import tqdm
import re
import datetime
from tqdm import tqdm
import datetime
from getpass import getpass
import xmltodict, json
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('precision', 2)

pd.options.display.float_format = '{:20,.2f}'.format
np.set_printoptions(suppress=False)
from pymongo import MongoClient

# Задание 4 урок

1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации

2)Сложить все новости в БД

In [30]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

def get_news_yandex(header):
    response = requests.get('https://yandex.ru/news', headers=header)
    dom = html.fromstring(response.text)
    news = []
    items = dom.xpath("//td[@class='stories-set__item']")
    for item in items:
        new = {}
        title = item.xpath(".//h2[@class='story__title']//text()")[0]     
        source = item.xpath(".//div[@class='story__date']/text()")[0][:-6] 
        link = 'https://yandex.ru'+item.xpath(".//h2[@class='story__title']//@href")[0]
        date = item.xpath(".//div[@class='story__date']/text()")[0][-5:]

        new['title'] = title
        new['source'] = source
        new['link'] = link
        new['date'] = date

        news.append(new)
        
    return(news)

In [31]:
findings = get_news_yandex(header)

In [32]:
# решил попробовать поскладывать в отдельные коллекции
def to_dataset(db_name, col_name, data):
    client = MongoClient('localhost', 27017)
    db = client['news']
    collection_name = f"{col_name}_{datetime.datetime.now().strftime(format='%Y-%m-%d-%H-%M-%S')}"
    collection = db[collection_name]
    for el in tqdm(data, total = len(data)):
        collection.insert_one(el)
    
    # проверяем
    display(pd.DataFrame(db[collection_name].find( {} )).head(10))

In [33]:
to_dataset('news', 'yandex', findings)

100%|█████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 853.35it/s]


,_id,title,source,link,date
0,5f05dfe2ec860e86a0e14a56,Минздрав одобрил препарат от коронавируса «Коронавир»,РБК,https://yandex.ru/news/story/Minzdrav_odobril_preparat_ot_koronavirusa_Koronavir--33232270632e99...,17:45
1,5f05dfe2ec860e86a0e14a57,В Подмосковье к 15 июля планируют открыть рестораны и пляжи,RT на русском,https://yandex.ru/news/story/V_Podmoskove_k_15_iyulya_planiruyut_otkryt_restorany_i_plyazhi--f89...,17:45
2,5f05dfe2ec860e86a0e14a58,В АТОР спрогнозировали сроки возобновления зарубежного туризма,Газета.Ru,https://yandex.ru/news/story/V_ATOR_sprognozirovali_sroki_vozobnovleniya_zarubezhnogo_turizma--8...,17:42
3,5f05dfe2ec860e86a0e14a59,Госдума 23 июля примет закон о приравнивании отчуждения к экстремизму,REGNUM,https://yandex.ru/news/story/Gosduma_23_iyulya_primet_zakon_o_priravnivanii_otchuzhdeniya_k_ehks...,17:42
4,5f05dfe2ec860e86a0e14a5a,Заксобрание утвердило кандидатуру Бельского на пост вице-губернатора,Gazeta.SPb,https://yandex.ru/news/story/Zaksobranie_utverdilo_kandidaturu_Belskogo_na_post_vice-gubernatora...,17:40
5,5f05dfe2ec860e86a0e14a5b,Двор «Никольских рядов» на Садовой улице открылся для посетителей,Собака.ru,https://yandex.ru/news/story/Dvor_Nikolskikh_ryadov_na_Sadovoj_ulice_otkrylsya_dlya_posetitelej-...,17:24
6,5f05dfe2ec860e86a0e14a5c,"Коллега Соколова рассказал, как он вел себя после убийства Ещенко",Gazeta.SPb,https://yandex.ru/news/story/Kollega_Sokolova_rasskazal_kak_on_vel_sebya_posle_ubijstva_Eshhenko...,17:10
7,5f05dfe2ec860e86a0e14a5d,В Петербурге дали разрешение на ввод второй очереди «Лахта Центра»,АиФ Санкт-Петербург,https://yandex.ru/news/story/V_Peterburge_dali_razreshenie_na_vvod_vtoroj_ocheredi_Lakhta_Centra...,15:37
8,5f05dfe2ec860e86a0e14a5e,Росздравнадзор: возгорания в больницах произошли не от аппаратов ИВЛ,Вести.Ru,https://yandex.ru/news/story/Roszdravnadzor_vozgoraniya_v_bolnicakh_proizoshli_ne_ot_apparatov_I...,17:34
9,5f05dfe2ec860e86a0e14a5f,Завершено расследование дела в отношении обокравшей банк кассирши,Известия,https://yandex.ru/news/story/Zaversheno_rassledovanie_dela_v_otnoshenii_obokravshej_bank_kassirs...,17:43


In [95]:
def get_news_lenta(header):
    response = requests.get('https://lenta.ru/', headers=header)
    dom = html.fromstring(response.text)
    news = []
    items = dom.xpath("//div[contains(@class, 'yellow-box__wrap')]//a")
    for item in items:
        new = {}
        title = item.xpath(".//text()")[0].replace('\xa0', ' ')     
        url = item.xpath("./@href")[0]
        if 'https' in url:
            continue
        time.sleep(1)
        link = 'https://lenta.ru'+url
        res = requests.get(link, headers=header)
        domlink = html.fromstring(res.text)
        date = domlink.xpath("//time[contains(@class, 'g-date')]/@datetime")[0]
        print(date, link)

        new['title'] = title
        new['link'] = link
        new['date'] = date

        news.append(new)
        
    return(news)

In [96]:
findings = get_news_lenta(header)

2020-07-08T15:16:00+03:00 https://lenta.ru/news/2020/07/08/progress/
2020-07-08T17:08:00+03:00 https://lenta.ru/news/2020/07/08/nalog/
2020-07-08T16:06:00+03:00 https://lenta.ru/news/2020/07/08/maya/
2020-07-08T12:46:00+03:00 https://lenta.ru/news/2020/07/08/biden_in_ussr/
2020-07-08T15:29:00+03:00 https://lenta.ru/news/2020/07/08/netlineek/
2020-07-08T13:11:00+03:00 https://lenta.ru/news/2020/07/08/peskov/
2020-07-08T14:55:00+03:00 https://lenta.ru/news/2020/07/08/rastorgli/
2020-07-08T11:34:00+03:00 https://lenta.ru/news/2020/07/08/proslushka/
2020-07-08T06:44:00+03:00 https://lenta.ru/news/2020/07/08/hiv/


In [97]:
findings

[{'title': 'Мишустин заявил об угрозе достижениям в борьбе с бедностью в России',
  'link': 'https://lenta.ru/news/2020/07/08/progress/',
  'date': '2020-07-08T15:16:00+03:00'},
 {'title': 'России предрекли самые большие потери из-за нового налога в Европе',
  'link': 'https://lenta.ru/news/2020/07/08/nalog/',
  'date': '2020-07-08T17:08:00+03:00'},
 {'title': 'Раскрыт сценарий возвращения международных перелетов в России',
  'link': 'https://lenta.ru/news/2020/07/08/maya/',
  'date': '2020-07-08T16:06:00+03:00'},
 {'title': 'Найдено свидетельство первого визита Байдена в СССР',
  'link': 'https://lenta.ru/news/2020/07/08/biden_in_ussr/',
  'date': '2020-07-08T12:46:00+03:00'},
 {'title': 'Российских школьников оставили без линеек 1 сентября',
  'link': 'https://lenta.ru/news/2020/07/08/netlineek/',
  'date': '2020-07-08T15:29:00+03:00'},
 {'title': 'Кремль ответил на главные вопросы по делу Сафронова',
  'link': 'https://lenta.ru/news/2020/07/08/peskov/',
  'date': '2020-07-08T13:11:0

In [98]:
to_dataset('news', 'lenta', findings)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 299.87it/s]


,_id,title,link,date
0,5f05e874ec860e86a0e14a97,Мишустин заявил об угрозе достижениям в борьбе с бедностью в России,https://lenta.ru/news/2020/07/08/progress/,2020-07-08T15:16:00+03:00
1,5f05e874ec860e86a0e14a98,России предрекли самые большие потери из-за нового налога в Европе,https://lenta.ru/news/2020/07/08/nalog/,2020-07-08T17:08:00+03:00
2,5f05e874ec860e86a0e14a99,Раскрыт сценарий возвращения международных перелетов в России,https://lenta.ru/news/2020/07/08/maya/,2020-07-08T16:06:00+03:00
3,5f05e874ec860e86a0e14a9a,Найдено свидетельство первого визита Байдена в СССР,https://lenta.ru/news/2020/07/08/biden_in_ussr/,2020-07-08T12:46:00+03:00
4,5f05e874ec860e86a0e14a9b,Российских школьников оставили без линеек 1 сентября,https://lenta.ru/news/2020/07/08/netlineek/,2020-07-08T15:29:00+03:00
5,5f05e874ec860e86a0e14a9c,Кремль ответил на главные вопросы по делу Сафронова,https://lenta.ru/news/2020/07/08/peskov/,2020-07-08T13:11:00+03:00
6,5f05e874ec860e86a0e14a9d,Украина расторгла меморандум с Россией о борьбе с терроризмом,https://lenta.ru/news/2020/07/08/rastorgli/,2020-07-08T14:55:00+03:00
7,5f05e874ec860e86a0e14a9e,Стало известно об «очень долгой слежке» ФСБ за советником главы «Роскосмоса»,https://lenta.ru/news/2020/07/08/proslushka/,2020-07-08T11:34:00+03:00
8,5f05e874ec860e86a0e14a9f,Третий человек в мире излечился от ВИЧ,https://lenta.ru/news/2020/07/08/hiv/,2020-07-08T06:44:00+03:00
